# 2편: 완전연결 신경망으로 FashionMNIST 분류하기

이 노트북에서는 다음을 학습합니다:
- FashionMNIST 데이터셋의 구조와 시각화
- DataLoader를 사용한 미니배치 학습
- 1계층 완전연결(FC) 신경망 구현
- ReLU 활성화 함수와 Dropout을 적용한 다계층 FC 신경망 구현
- 테스트 정확도 평가 및 혼동 행렬(Confusion Matrix) 시각화

## 1. FashionMNIST 데이터셋

FashionMNIST는 Zalando에서 공개한 패션 아이템 이미지 데이터셋입니다.
MNIST 숫자 데이터셋과 동일한 포맷(28x28 그레이스케일)이지만, 의류 아이템을 분류하는 더 어려운 과제입니다.

| 항목 | 내용 |
|---|---|
| 이미지 크기 | 28 x 28 그레이스케일 |
| 학습 데이터 | 60,000장 |
| 테스트 데이터 | 10,000장 |
| 클래스 | 10가지 (T-shirt, Trouser, Pullover 등) |

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np
import time

# FashionMNIST 다운로드
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# 클래스 이름 매핑
class_names = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]

print(f"학습 데이터: {len(training_data)}장")
print(f"테스트 데이터: {len(test_data)}장")
print(f"클래스 수: {len(class_names)}")
print(f"이미지 shape: {training_data[0][0].shape}")

In [ ]:
# 샘플 이미지 시각화 (2x5 그리드)
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    image, label = training_data[i]
    ax.imshow(image.squeeze(), cmap="gray")
    ax.set_title(class_names[label], fontsize=10)
    ax.axis("off")
plt.suptitle("FashionMNIST Samples", fontsize=14)
plt.tight_layout()
plt.show()

## 2. DataLoader로 배치 학습 준비

PyTorch의 데이터 파이프라인은 두 핵심 클래스로 구성됩니다:

- **Dataset**: 개별 샘플(이미지, 레이블)에 인덱스로 접근하는 인터페이스
- **DataLoader**: Dataset을 감싸서 미니배치 단위로 데이터를 공급하는 반복자

**batch_size=64**로 설정합니다. 배치 크기는 학습 안정성과 메모리 사용량 사이의 균형을 결정합니다.
작은 배치는 노이즈가 많지만 일반화 성능이 좋고, 큰 배치는 안정적이지만 메모리를 많이 사용합니다.

**Flatten**: 28x28 이미지를 784차원 벡터로 펼쳐야 완전연결 계층에 입력할 수 있습니다.
`x.view(-1, 784)` 또는 `x.flatten(1)`로 변환합니다.

In [ ]:
batch_size = 64

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

print(f"전체 학습 데이터: {len(training_data)}장")
print(f"배치 크기: {batch_size}")
print(f"학습 배치 수: {len(train_loader)} (= ceil({len(training_data)} / {batch_size}))")
print(f"테스트 배치 수: {len(test_loader)}")

# 첫 번째 배치 확인
sample_images, sample_labels = next(iter(train_loader))
print(f"\n배치 이미지 shape: {sample_images.shape}")
print(f"  -> {sample_images.shape[0]}장 x {sample_images.shape[2]}x{sample_images.shape[3]} 이미지 x {sample_images.shape[1]} 채널")
print(f"배치 레이블 shape: {sample_labels.shape}")

# Flatten 변환 시연
flattened = sample_images.view(-1, 28 * 28)
print(f"\nFlatten 전: {sample_images.shape}")
print(f"Flatten 후: {flattened.shape}  (28 x 28 = 784차원 벡터)")

## 3. 1계층 완전연결 신경망

가장 단순한 구조로, 입력을 바로 출력 클래스에 매핑합니다.

```
입력 (28x28) -> Flatten (784) -> Linear(784, 10) -> 출력 (10 클래스)
```

**파라미터 수 계산**:
- 가중치: 784 x 10 = 7,840개
- 편향(bias): 10개
- **합계: 7,850개**

단순하지만, 선형 변환만으로는 복잡한 패턴을 학습하기 어렵습니다.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"사용 디바이스: {device}")


class SingleLayerNet(nn.Module):
    """1계층 완전연결 신경망: 784 -> 10"""

    def __init__(self):
        super().__init__()
        self.fc_out = nn.Linear(28 * 28, 10)

    def forward(self, x):
        return self.fc_out(x)


fc_single = SingleLayerNet().to(device)
print(fc_single)

single_params = sum(p.numel() for p in fc_single.parameters())
print(f"\n총 파라미터 수: {single_params:,}개")

In [ ]:
# 1계층 모델 학습
optimizer_single = torch.optim.Adam(fc_single.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss().to(device)

num_epochs = 15
single_costs = []
single_times = []

fc_single.train()
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    num_batches = len(train_loader)

    for images, labels in train_loader:
        images = images.view(-1, 784).to(device)
        labels = labels.to(device)

        optimizer_single.zero_grad()
        outputs = fc_single(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_single.step()

        running_loss += loss.item() / num_batches

    elapsed = time.time() - start_time
    single_costs.append(running_loss)
    single_times.append(elapsed)
    print(f"Epoch {epoch + 1:02d}/{num_epochs} | loss = {running_loss:.6f} | time = {elapsed:.2f}s")

print(f"\n평균 에포크 시간: {np.mean(single_times):.2f}s")

## 4. 다계층 완전연결 신경망

1계층 모델의 한계를 극복하기 위해 **은닉층(hidden layer)**을 추가합니다.
핵심 차이점은 계층 사이에 **ReLU 활성화 함수**와 **Dropout**을 적용하는 것입니다.

```
입력(784) -> Linear(784, 512) -> ReLU -> Dropout(0.3)
          -> Linear(512, 256) -> ReLU -> Dropout(0.3)
          -> Linear(256, 10)  -> 출력(10 클래스)
```

**ReLU (Rectified Linear Unit)**: `max(0, x)` - 비선형성을 도입하여 복잡한 패턴을 학습할 수 있게 합니다.
활성화 함수 없이 선형 계층만 쌓으면 결국 하나의 선형 변환과 동일합니다.

**Dropout(0.3)**: 학습 시 뉴런의 30%를 무작위로 비활성화하여 과적합을 방지합니다.

**파라미터 수 계산**:
- 1층: 784 x 512 + 512 = 401,920
- 2층: 512 x 256 + 256 = 131,328
- 3층: 256 x 10 + 10 = 2,570
- **합계: 535,818개** (1계층 대비 약 68배)

In [ ]:
class MultiLayerNet(nn.Module):
    """다계층 완전연결 신경망: 784 -> 512 -> 256 -> 10 (ReLU + Dropout)"""

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


fc_multi = MultiLayerNet().to(device)
print(fc_multi)

multi_params = sum(p.numel() for p in fc_multi.parameters())
print(f"\n총 파라미터 수: {multi_params:,}개")
print(f"1계층 대비 {multi_params / single_params:.0f}배")

# 계층별 파라미터 수 상세
print("\n[계층별 파라미터 수]")
for name, param in fc_multi.named_parameters():
    print(f"  {name:20s} -> {param.numel():>10,}개  {list(param.shape)}")

In [ ]:
# 다계층 모델 학습
optimizer_multi = torch.optim.Adam(fc_multi.parameters(), lr=0.001)

multi_costs = []
multi_times = []

fc_multi.train()
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    num_batches = len(train_loader)

    for images, labels in train_loader:
        images = images.view(-1, 784).to(device)
        labels = labels.to(device)

        optimizer_multi.zero_grad()
        outputs = fc_multi(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_multi.step()

        running_loss += loss.item() / num_batches

    elapsed = time.time() - start_time
    multi_costs.append(running_loss)
    multi_times.append(elapsed)
    print(f"Epoch {epoch + 1:02d}/{num_epochs} | loss = {running_loss:.6f} | time = {elapsed:.2f}s")

print(f"\n평균 에포크 시간: {np.mean(multi_times):.2f}s")

## 5. 테스트 정확도 평가

학습이 끝난 모델을 **테스트 데이터**로 평가합니다.
테스트 데이터는 학습 과정에서 한 번도 사용하지 않은 데이터이므로, 모델의 일반화 성능을 측정할 수 있습니다.

평가 시에는 반드시 `model.eval()`과 `torch.no_grad()`를 사용합니다:
- `model.eval()`: Dropout 비활성화, BatchNorm을 추론 모드로 전환
- `torch.no_grad()`: 그래디언트 계산을 중단하여 메모리 절약

In [ ]:
def evaluate(model, dataloader):
    """모델을 평가하여 정확도, 예측값, 실제값을 반환한다."""
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.view(-1, 784).to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    return accuracy, np.array(all_preds), np.array(all_labels)


# 두 모델 평가
single_acc, single_preds, single_labels = evaluate(fc_single, test_loader)
multi_acc, multi_preds, multi_labels = evaluate(fc_multi, test_loader)

print(f"1계층 모델 테스트 정확도: {single_acc:.4f} ({single_acc * 100:.2f}%)")
print(f"다계층 모델 테스트 정확도: {multi_acc:.4f} ({multi_acc * 100:.2f}%)")

# 클래스별 정확도 계산 (다계층 모델)
per_class_correct = np.zeros(10)
per_class_total = np.zeros(10)
for pred, label in zip(multi_preds, multi_labels):
    per_class_total[label] += 1
    if pred == label:
        per_class_correct[label] += 1

per_class_acc = per_class_correct / per_class_total

# 클래스별 정확도 수평 막대 차트
fig, ax = plt.subplots(figsize=(10, 5))
colors = plt.cm.RdYlGn(per_class_acc)  # 정확도에 따라 색상 변화
bars = ax.barh(range(10), per_class_acc * 100, color=colors, edgecolor="gray", linewidth=0.5)
ax.set_yticks(range(10))
ax.set_yticklabels(class_names)
ax.set_xlabel("Accuracy (%)")
ax.set_title(f"Per-Class Accuracy (MultiLayerNet, Overall: {multi_acc * 100:.1f}%)")
ax.set_xlim(0, 100)
ax.axvline(x=multi_acc * 100, color="red", linestyle="--", alpha=0.7, label=f"Average: {multi_acc * 100:.1f}%")

# 막대 끝에 수치 표시
for i, (bar, acc) in enumerate(zip(bars, per_class_acc)):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height() / 2,
            f"{acc * 100:.1f}%", va="center", fontsize=9)

ax.legend()
ax.grid(axis="x", alpha=0.3)
plt.tight_layout()
plt.show()

# 혼동 행렬 (Confusion Matrix) 계산 및 시각화
confusion_mat = np.zeros((10, 10), dtype=int)
for pred, label in zip(multi_preds, multi_labels):
    confusion_mat[label][pred] += 1

fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(confusion_mat, cmap="Blues")

ax.set_xticks(range(10))
ax.set_yticks(range(10))
ax.set_xticklabels(class_names, rotation=45, ha="right", fontsize=9)
ax.set_yticklabels(class_names, fontsize=9)
ax.set_xlabel("Predicted")
ax.set_ylabel("Actual")
ax.set_title("Confusion Matrix (MultiLayerNet)")

# 셀에 수치 표시
for i in range(10):
    for j in range(10):
        value = confusion_mat[i][j]
        color = "white" if value > confusion_mat.max() * 0.5 else "black"
        ax.text(j, i, str(value), ha="center", va="center", color=color, fontsize=8)

fig.colorbar(im, ax=ax, shrink=0.8)
plt.tight_layout()
plt.show()

## 6. 1계층 vs 다계층 비교

두 모델의 학습 과정과 성능을 비교합니다.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epochs_range = range(1, num_epochs + 1)

# Loss 비교
axes[0].plot(epochs_range, single_costs, "o-", markersize=4,
             label=f"SingleLayerNet ({single_params:,} params)")
axes[0].plot(epochs_range, multi_costs, "s-", markersize=4,
             label=f"MultiLayerNet ({multi_params:,} params)")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[0].set_title("Training Loss")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 에포크별 학습 시간 비교
axes[1].plot(epochs_range, single_times, "o-", markersize=4, label="SingleLayerNet")
axes[1].plot(epochs_range, multi_times, "s-", markersize=4, label="MultiLayerNet")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Time (seconds)")
axes[1].set_title("Training Time per Epoch")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 요약 테이블
print(f"{'='*65}")
print(f"{'항목':20s} {'SingleLayerNet':>18s} {'MultiLayerNet':>18s}")
print(f"{'-'*65}")
print(f"{'파라미터 수':20s} {single_params:>18,} {multi_params:>18,}")
print(f"{'최종 학습 loss':20s} {single_costs[-1]:>18.6f} {multi_costs[-1]:>18.6f}")
print(f"{'테스트 정확도':20s} {single_acc * 100:>17.2f}% {multi_acc * 100:>17.2f}%")
print(f"{'평균 에포크 시간':20s} {np.mean(single_times):>17.2f}s {np.mean(multi_times):>17.2f}s")
print(f"{'='*65}")

## 완전연결 신경망의 한계

현재는 28x28의 아주 작은 그레이스케일 이미지를 다루고 있지만,
실제 환경에서는 훨씬 큰 이미지를 처리해야 합니다.

**4K 이미지 (3840 x 2160 x 3 = 24,883,200차원)를 FC 레이어에 입력하면?**
- 첫 번째 은닉층이 512개 뉴런만 가져도: 24,883,200 x 512 = **약 127억 개의 파라미터**
- float32 기준 약 **47GB의 메모리**가 필요
- 학습이 사실상 불가능

**근본적인 문제**: 완전연결 신경망은 이미지를 1차원 벡터로 펼치기 때문에
**공간적 구조(인접 픽셀 간의 관계)**를 전혀 활용하지 못합니다.

예를 들어, 이미지에서 "가장자리"나 "질감" 같은 지역적 패턴은
인접한 픽셀들의 관계에서 나오는데, FC 레이어는 모든 픽셀을 독립적으로 취급합니다.

**다음 노트북에서 CNN(합성곱 신경망)**으로 이 문제를 해결합니다.
CNN은 작은 필터가 이미지 위를 슬라이딩하며 지역 패턴을 추출하므로,
파라미터 수를 크게 줄이면서도 공간 정보를 보존합니다.